In [1]:
# cp /data/train.zip ~/

In [2]:
# cp /data/test.zip ~/

In [3]:
# !unzip test.zip

In [4]:
# !unzip train.zip

In [5]:
# cp /data/pretrained_resnext.pt ~/

In [6]:
# cp /data/test.csv ~/

## Step1: Import modules

In [7]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics
import torch
import torch.nn as nn
import torch.optim as optim



import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split

from PIL import Image # view image
import time # 
import copy
import os
import shutil

plt.ion()   # interactive mode

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using => ", device)

root_dir = ""
print("The data lies here => ", root_dir)
if not os.path.exists(root_dir+'train/'):
    os.makedirs(root_dir+'train/')
if not os.path.exists(root_dir+'test/'):
    os.makedirs(root_dir+'/test')
    
model_dir = root_dir + 'model/'
print("The best model lies here => ", model_dir)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
ckp_dir = root_dir + 'checkpoint/'
print("Lastest model checkpoint lies here => ", ckp_dir)
if not os.path.exists(ckp_dir):
    os.makedirs(ckp_dir)

We're using =>  cuda
The data lies here =>  
The best model lies here =>  model/
Lastest model checkpoint lies here =>  checkpoint/


## Step2: Prepare dataset for training

In [ ]:
# Define transforms
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(25),
#         transforms.RandomGrayscale(p=0.2),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [10]:
# Initialize Datasets
# Train data floders with be divied into train + val
product_dataset = datasets.ImageFolder(root=root_dir+"train",
                                      transform=image_transforms["train"])
class_names = product_dataset.classes
product_dataset

Dataset ImageFolder
    Number of datapoints: 105392
    Root location: train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               ColorJitter(brightness=None, contrast=None, saturation=None, hue=None)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=(-25, 25), resample=False, expand=False)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [11]:
demo_size = 0.02 # 2K
demo_len = int(demo_size*len(product_dataset))
tiny_dataset, _ = random_split(product_dataset, [demo_len, len(product_dataset)-demo_len])
tiny_dataset

In [12]:
demo = False
if demo == True:
    print("we're using demo small dataset.")
    input_dataset = tiny_dataset
else:
    print("we're using normal big dataset")
    input_dataset = product_dataset

we're using normal big dataset


In [13]:
# Get train and validation samples
# SubsetRandomSampler
input_dataset_size = len(input_dataset)
input_dataset_indices = list(range(input_dataset_size))

np.random.shuffle(input_dataset_indices)
val_split_index = int(np.floor(0.2 * input_dataset_size))

train_idx, val_idx = input_dataset_indices[val_split_index:],\
input_dataset_indices[:val_split_index]

# train, val samplers
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)
# train, val dataloaders
batch_size = 16
dataloaders = {
    "train": DataLoader(input_dataset, shuffle=False, 
                           batch_size=batch_size, sampler=train_sampler),
    "val": DataLoader(input_dataset, shuffle=False, 
                        batch_size=8, sampler=val_sampler)
}
dataset_sizes = {'train': input_dataset_size-val_split_index, 
                 'val': val_split_index}

In [14]:
dataset_sizes

{'train': 84314, 'val': 21078}

In [15]:
!nvidia-smi

Sat Jul  4 03:48:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3E:00.0 Off |                    0 |
| N/A   42C    P0    61W / 280W |     12MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Step3: Data Augmentation
1. mixcut
2. data generator

## Step4: Load pre-trained model & model save
1. EfficentNet trained on Imagenet. pytorch efficientnet
2. Freeze weight of previous layers
3. Change output layers from 1000(ImageNet) to 42 
4. save trained models to Google drive

In [16]:
#using efficientnet model based transfer learning
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # freeze trained model
        self.model_ft = torch.load('pretrained_resnext.pt')
        # for param in self.pretrained.parameters():
        #   param.requires_grad = False
        self.l1 = nn.Linear(1000 , 256)
        self.dropout = nn.Dropout(0.75)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(256, 42)

    def forward(self, input):
        x = self.model_ft(input)
        x = x.view(x.size(0),-1)
        x = self.dropout(self.relu(self.l1(x)))
        x = self.l2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = Classifier().to(device)

# check our model is on right device
print("Our classifier is on the device => ", list(classifier.parameters())[0].device)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torc

Our classifier is on the device =>  cuda:0


In [17]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

def load_ckp(checkpoint_fpath, model):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
#     optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # initialize valid_acc_best from checkpoint to valid_acc_best
    valid_acc_best = checkpoint['valid_acc_best']
    # return model, optimizer, epoch value, min validation loss 
    return model, checkpoint['epoch'], valid_loss_min, valid_acc_best.item()

In [18]:
model, epoch, _, best_acc = load_ckp('best_model_10am.pt', classifier)

## Step5: Loss function, optimizer

In [19]:
criterion = nn.CrossEntropyLoss()
# observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.00001)
# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [20]:
torch.cuda.empty_cache()

In [21]:
!nvidia-smi

Sat Jul  4 03:48:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3E:00.0 Off |                    0 |
| N/A   41C    P0    61W / 280W |   1519MiB / 16160MiB |     20%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Step6: Train
1.Hyperparameter  
2.Use ResNext101 for fine tuning

In [22]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, checkpoint_path=None, best_model_path=None):
    since = time.time()
#     if os.path.exists('best/current_checkpoint.pt'):
#         model, epoch, _, best_acc = load_ckp('checkpoint/current_checkpoint.pt', model, optimizer)
#     else:
#         best_acc = 0.0
# #     best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        torch.cuda.empty_cache()
        !nvidia-smi
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            CNT = 0
            # Iterate over data.
            for batch_idx, (inputs, labels) in enumerate(dataloaders[phase]):
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                # track history if only in train
                # with torch.set_grad_enabled(phase == 'train'):
                #     outputs = model(inputs)
                #     _, preds = torch.max(outputs, 1)
                #     loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                batchsize = inputs.size(0)
                running_loss += loss.item() * batchsize
                running_corrects += torch.sum(preds == labels.data)
                CNT += 1
                if batch_idx % 100 == 0:
                    print('   {}:  [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tAcc: {:.6f}'.format(phase,
                        CNT*batchsize, dataset_sizes[phase], 100. * CNT/len(dataloaders[phase]),
                        running_loss/(CNT*batchsize), running_corrects.double()/(CNT*batchsize)))
#                 if batch_idx == 300:
#                     break
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # create checkpoint variable and add important data
            checkpoint = {
                'epoch': epoch + 1,
                'valid_loss_min': epoch_loss,
                'valid_acc_best': epoch_acc,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
                }            
            # deep copy and save the model
            if phase == 'val':
                if epoch_acc >= best_acc:
                    save_ckp(checkpoint, True, checkpoint_path, best_model_path)
                    best_acc = epoch_acc
#                     best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    save_ckp(checkpoint, False, checkpoint_path, best_model_path)
            print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # return the best model weights
    model, _, _, _ = load_ckp('model/best_model.pt', model, optimizer)
    return model

In [23]:
model_final = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=50, checkpoint_path=ckp_dir+'current_checkpoint.pt',
                       best_model_path=model_dir+'best_model.pt')

Epoch 0/49
----------
Sat Jul  4 03:48:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3E:00.0 Off |                    0 |
| N/A   41C    P0    61W / 280W |   1519MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                      

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


   train:  [16/84314 (0%)]	Loss: 0.543112 	Acc: 0.812500
   train:  [1616/84314 (2%)]	Loss: 0.736437 	Acc: 0.798886
   train:  [3216/84314 (4%)]	Loss: 0.704434 	Acc: 0.814988
   train:  [4816/84314 (6%)]	Loss: 0.681513 	Acc: 0.821221
   train:  [6416/84314 (8%)]	Loss: 0.670609 	Acc: 0.822007
   train:  [8016/84314 (10%)]	Loss: 0.660338 	Acc: 0.825100
   train:  [9616/84314 (11%)]	Loss: 0.655508 	Acc: 0.825603
   train:  [11216/84314 (13%)]	Loss: 0.656891 	Acc: 0.826854
   train:  [12816/84314 (15%)]	Loss: 0.645355 	Acc: 0.829744
   train:  [14416/84314 (17%)]	Loss: 0.633824 	Acc: 0.831853
   train:  [16016/84314 (19%)]	Loss: 0.636788 	Acc: 0.831231
   train:  [17616/84314 (21%)]	Loss: 0.631507 	Acc: 0.832312
   train:  [19216/84314 (23%)]	Loss: 0.628688 	Acc: 0.832379
   train:  [20816/84314 (25%)]	Loss: 0.625442 	Acc: 0.833061
   train:  [22416/84314 (27%)]	Loss: 0.625628 	Acc: 0.833378
   train:  [24016/84314 (28%)]	Loss: 0.628590 	Acc: 0.833611
   train:  [25616/84314 (30%)]	Loss: 0

KeyboardInterrupt: 

In [ ]:
!nvidia-smi